# MetroPT-3 dataset quick profile

Quick look at the MetroPT-3 telemetry used by the IsolationForest helper:
- shape, columns, and timestamp coverage
- sampling cadence statistics (to verify ~1 Hz)
- per-column numeric stats
- day-level coverage

Update `DATA_PATH` below if your CSV lives elsewhere.


In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
from IPython.display import display

DATA_PATH = Path('MetroPT3.csv')  # change if needed

def infer_timestamp_column(df: pd.DataFrame):
    preferred = ['timestamp', 'time', 'datetime', 'date', 'Date', 'Timestamp', 'Time']
    for c in preferred:
        if c in df.columns:
            return c
    for c in df.columns:
        try:
            pd.to_datetime(df[c])
            return c
        except Exception:
            continue
    raise ValueError('Could not infer timestamp column; please set it manually.')

df_raw = pd.read_csv(DATA_PATH)
ts_col = infer_timestamp_column(df_raw)
df_raw[ts_col] = pd.to_datetime(df_raw[ts_col], errors='coerce')
df = df_raw.dropna(subset=[ts_col]).sort_values(ts_col).reset_index(drop=True).set_index(ts_col)
df.head()


,Unnamed: 0,TP2,TP3,H1,DV_pressure,Reservoirs,Oil_temperature,Motor_current,COMP,DV_eletric,Towers,MPG,LPS,Pressure_switch,Oil_level,Caudal_impulses
timestamp,,,,,,,,,,,,,,,,
2020-02-01 00:00:00,0,-0.012,9.358,9.340,-0.024,9.358,53.600,0.0400,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0
2020-02-01 00:00:10,10,-0.014,9.348,9.332,-0.022,9.348,53.675,0.0400,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0
2020-02-01 00:00:19,20,-0.012,9.338,9.322,-0.022,9.338,53.600,0.0425,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0
2020-02-01 00:00:29,30,-0.012,9.328,9.312,-0.022,9.328,53.425,0.0400,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0
2020-02-01 00:00:39,40,-0.012,9.318,9.302,-0.022,9.318,53.475,0.0400,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0


In [2]:
bin_cols = [
    c for c in df.select_dtypes(include=[np.number]).columns
    if set(df[c].dropna().unique()) <= {0, 1}
]

overview = pd.DataFrame({
    'value': [
        len(df),
        df.shape[1],
        df.select_dtypes(include=[np.number]).shape[1],
        len(bin_cols),
    ]
}, index=['rows', 'columns', 'numeric_columns', 'binary_0_1_columns'])
overview


,value
rows,1516948
columns,16
numeric_columns,16
binary_0_1_columns,8


In [3]:
# Dtype and null summary
dtype_nulls = pd.DataFrame({
    'dtype': df.dtypes.astype(str),
    'non_null': df.notnull().sum(),
    'nulls': df.isnull().sum(),
})
dtype_nulls


,dtype,non_null,nulls
Unnamed: 0,int64,1516948,0
TP2,float64,1516948,0
TP3,float64,1516948,0
H1,float64,1516948,0
DV_pressure,float64,1516948,0
Reservoirs,float64,1516948,0
Oil_temperature,float64,1516948,0
Motor_current,float64,1516948,0
COMP,float64,1516948,0
DV_eletric,float64,1516948,0


In [4]:
# Time coverage and sampling cadence (seconds)
time_coverage = pd.DataFrame({
    'start': [df.index.min()],
    'end': [df.index.max()],
    'span': [df.index.max() - df.index.min()],
    'span_days': [(df.index.max() - df.index.min()).total_seconds() / 86400],
})
time_coverage

deltas = df.index.to_series().diff().dropna().dt.total_seconds()
delta_stats = deltas.describe(percentiles=[0.5, 0.9, 0.95, 0.99]).to_frame('seconds')
delta_top = deltas.round(3).value_counts().head(10).to_frame('count')
delta_top['%'] = 100 * delta_top['count'] / delta_top['count'].sum()
delta_top.index.name = 'delta_seconds'

display(delta_stats)
display(delta_top)


,seconds
count,1.516947e+06
mean,1.214122e+01
std,3.141073e+02
min,8.000000e+00
50%,1.000000e+01
90%,1.000000e+01
95%,1.000000e+01
99%,1.200000e+01
max,1.729180e+05


,count,%
delta_seconds,,
10.0,1337521,88.191900
9.0,128277,8.458179
12.0,38321,2.526765
13.0,7988,0.526703
11.0,4471,0.294804
21.0,10,0.000659
19.0,5,0.000330
22.0,4,0.000264
14.0,3,0.000198


In [5]:
# Per-column stats (numeric only)
num_cols = df.select_dtypes(include=[np.number])
num_summary = num_cols.describe(percentiles=[0.01, 0.05, 0.25, 0.5, 0.75, 0.95, 0.99]).T
num_summary


,count,mean,std,min,1%,5%,25%,50%,75%,95%,99%,max
Unnamed: 0,1516948.0,7.584735e+06,4.379053e+06,0.000,151694.700,758473.5000,3792367.500,7584735.000,1.137710e+07,1.441100e+07,1.501778e+07,1.516947e+07
TP2,1516948.0,1.367826e+00,3.250930e+00,-0.032,-0.024,-0.0160,-0.014,-0.012,-1.000000e-02,9.590000e+00,1.035200e+01,1.067600e+01
TP3,1516948.0,8.984611e+00,6.390951e-01,0.730,7.876,8.1380,8.492,8.960,9.492000e+00,9.980000e+00,1.014000e+01,1.030200e+01
H1,1516948.0,7.568155e+00,3.333200e+00,-0.036,-0.024,-0.0140,8.254,8.784,9.374000e+00,9.916000e+00,1.011400e+01,1.028800e+01
DV_pressure,1516948.0,5.595619e-02,3.824015e-01,-0.032,-0.026,-0.0240,-0.022,-0.020,-1.800000e-02,-8.000000e-03,2.112000e+00,9.844000e+00
Reservoirs,1516948.0,8.985233e+00,6.383070e-01,0.712,7.876,8.1400,8.494,8.960,9.492000e+00,9.978000e+00,1.013800e+01,1.030000e+01
Oil_temperature,1516948.0,6.264418e+01,6.516261e+00,15.400,48.825,52.3250,57.775,62.700,6.725000e+01,7.352500e+01,7.617500e+01,8.905000e+01
Motor_current,1516948.0,2.050171e+00,2.302053e+00,0.020,0.035,0.0375,0.040,0.045,3.807500e+00,5.987500e+00,6.187500e+00,9.295000e+00
COMP,1516948.0,8.369568e-01,3.694052e-01,0.000,0.000,0.0000,1.000,1.000,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
DV_eletric,1516948.0,1.606106e-01,3.671716e-01,0.000,0.000,0.0000,0.000,0.000,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00


In [6]:
# Rows per day distribution (summary stats)
day_counts = df.index.normalize().value_counts()
day_stats = day_counts.describe(percentiles=[0.5, 0.9, 0.95, 0.99]).to_frame("rows_per_day")
day_stats  # includes count, mean, std, min, 50% (median), 90%, 95%, 99%, max


,rows_per_day
count,212.000000
mean,7155.415094
std,1833.005130
min,210.000000
50%,7435.500000
90%,8716.000000
95%,8716.000000
99%,8717.000000
max,8717.000000


In [7]:
# Coverage summary in hours/days/weeks/months
from pandas import Timedelta
duration = df.index.max() - df.index.min()
span_hours = duration / Timedelta(hours=1)
span_days = duration / Timedelta(days=1)
span_weeks = duration / Timedelta(weeks=1)
# Approx month span from start/end (continuous), and unique calendar months covered
span_months_continuous = span_days / 30.4375  # rough avg month length
unique_days = df.index.normalize().nunique()
unique_weeks = df.index.to_period('W').nunique()
unique_months = df.index.to_period('M').nunique()
coverage = pd.DataFrame({
    'value': [
        span_hours,
        span_days,
        span_weeks,
        span_months_continuous,
        unique_days,
        unique_weeks,
        unique_months,
    ],
}, index=[
    'span_hours',
    'span_days',
    'span_weeks',
    'span_months_continuous',
    'unique_days',
    'unique_weeks',
    'unique_months',
])
coverage


,value
span_hours,5115.997222
span_days,213.166551
span_weeks,30.452364
span_months_continuous,7.003419
unique_days,212.000000
unique_weeks,32.000000
unique_months,8.000000


In [8]:
# Predicted phase vs operation_phase (point-wise metrics from prediction CSV)
import numpy as np
import pandas as pd

pred = pd.read_csv('metropt3_iforest_pred.csv', parse_dates=['timestamp'])
pred = pred.set_index('timestamp')

required_cols = {'predicted_phase', 'operation_phase'}
missing = required_cols - set(pred.columns)
if missing:
    raise ValueError(f'Missing columns in prediction CSV: {missing}')

mask = pred['operation_phase'].isin([0, 1]) & pred['predicted_phase'].notna()
y_true = (pred.loc[mask, 'operation_phase'] == 1).astype(int)
y_pred = pred.loc[mask, 'predicted_phase'].astype(int)

tp = int(((y_true == 1) & (y_pred == 1)).sum())
fp = int(((y_true == 0) & (y_pred == 1)).sum())
fn = int(((y_true == 1) & (y_pred == 0)).sum())
tn = int(((y_true == 0) & (y_pred == 0)).sum())

precision = tp / (tp + fp) if (tp + fp) else 0.0
recall = tp / (tp + fn) if (tp + fn) else 0.0
f1 = (2 * precision * recall / (precision + recall)) if (precision + recall) else 0.0
accuracy = (tp + tn) / (tp + fp + fn + tn) if (tp + fp + fn + tn) else 0.0

counts = pd.DataFrame(
    {'count': [tp, fp, fn, tn]},
    index=['TP', 'FP', 'FN', 'TN'],
)
ratios = pd.DataFrame(
    {'ratio': [precision, recall, f1, accuracy]},
    index=['precision', 'recall', 'f1', 'accuracy'],
)

display(counts)
display(ratios)


,count
TP,29
FP,12969
FN,10431
TN,1444898


,ratio
precision,0.002231
recall,0.002772
f1,0.002473
accuracy,0.984063


In [9]:
# Event-level metrics using predicted_phase alarm intervals
import numpy as np
import pandas as pd

from anomaly_iforest_helper import DEFAULT_METROPT_WINDOWS, PRE_MAINTENANCE_MINUTES
from iforest_metrics import _alarm_intervals_from_mask, _risk_scores

pred = pd.read_csv('metropt3_iforest_pred.csv', parse_dates=['timestamp'])
pred = pred.set_index('timestamp')

required_cols = {'predicted_phase'}
missing = required_cols - set(pred.columns)
if missing:
    raise ValueError(f'Missing columns in prediction CSV: {missing}')

alarm_mask = pred['predicted_phase'].fillna(0).astype(bool)
alarm_intervals = _alarm_intervals_from_mask(alarm_mask)

failure_windows = [(pd.to_datetime(s), pd.to_datetime(e)) for s, e, *_ in DEFAULT_METROPT_WINDOWS]
horizon = pd.to_timedelta(float(max(0.0, PRE_MAINTENANCE_MINUTES)), unit='m')
stats = _risk_scores(alarm_intervals, failure_windows, horizon)

tp = int(stats['tp'])
fp = int(stats['fp'])
fn = int(stats['fn'])
# TN is not defined at event-level because there is no explicit list of negative events.
tn = np.nan

precision = stats['precision']
recall = stats['recall']
f1 = stats['f1']
# Accuracy depends on TN, so it is not defined for event-level metrics.
accuracy = np.nan

counts = pd.DataFrame(
    {'count': [tp, fp, fn, tn]},
    index=['TP', 'FP', 'FN', 'TN'],
)
ratios = pd.DataFrame(
    {'ratio': [precision, recall, f1, accuracy]},
    index=['precision', 'recall', 'f1', 'accuracy'],
)

display(counts)
display(ratios)


,count
TP,15.0
FP,24.0
FN,6.0
TN,NaN


,ratio
precision,0.384615
recall,0.714286
f1,0.500000
accuracy,NaN
